Here, we compare the Hopf density of two gauges. I will use the [analytical results of Konstantin Y. Guslienko](https://doi.org/10.1016/j.chaos.2023.113840).

The stereographic projection ansatz is

\begin{equation}
    m_z = 1 - 2\tanh^2(\eta),
\end{equation}

with

\begin{equation}
\eta = \ln \left( \frac{d_1}{d_2} \right),
\end{equation}

where $d_1^2 = (\rho + a)^2 + z^2$ and $d_2^2 = (\rho - a)^2 + z^2$ for cylindrical coordinates $(\rho, \phi, z)$, see [the Wikipedia article on toroidal coordinates](https://en.wikipedia.org/wiki/Toroidal_coordinates). $a = 1$ is a scale factor. The $x$- and $y$-components of the magnetisation are then

\begin{align}
    m_x &= \sqrt{1 - m_z^2} \cos(\phi + \beta), \\
    m_y &= \sqrt{1 - m_z^2} \sin(\phi + \beta), \\
\end{align}

where

\begin{equation}
    \beta = \operatorname{sign}(z) \arccos \left( \frac{\rho^2 + z^2 - a^2}{\sqrt{(\rho^2 + z^2 - a^2)^2 + 4a^2z^2}} \right).
\end{equation}

One possible gauge in which to calculate the Hopf index is

\begin{align}
	A_x(x, y, z) &= \int_{-\infty}^y \mathrm{d}y' F_z(x, y', z), \\
	A_y(x, y, z) &= 0, \\
	A_z(x, y, z) &= -\int_{-\infty}^y \mathrm{d}y' F_x(x, y', z).
\end{align}

In the Coulomb gauge, however, a different $\boldsymbol{A}$ is used, the explicit form of which is given in [Konstantin Y. Guslienko's paper](https://doi.org/10.1016/j.chaos.2023.113840).

In [12]:
%display latex
import matplotlib.pyplot as plt
import numpy as np

# Declare cylindrical coordinate system
x = var('x')
y = var('y')
rho = sqrt(x^2 + y^2)
phi = arctan2(y, x)
z   = var('z')

# Scaling factor
a = 1.

d_1 = sqrt((rho + a)^2 + z^2)
d_2 = sqrt((rho - a)^2 + z^2)
eta = ln(d_1 / d_2)
beta = sign(z) * arccos((rho^2 + z^2 - a^2) / sqrt((rho^2 + z^2 - a^2)^2 + 4*a^2*z^2))

mz = 1 - 2 * tanh(eta)^2
mx = sqrt(1 - mz^2) * cos(phi + beta)
my = sqrt(1 - mz^2) * sin(phi + beta)

m = vector([mx, my, mz])

I first calculate the Hopf index in the Coulomb gauge.

In [2]:
# "Hack" using chain rule as Sage doesn't let you explicitly calculate dbeta / drho etc.
dbeta_drho = diff(beta, x) / diff(rho, x) + diff(beta, y) / diff(rho, y)
dmz_drho = diff(mz, x) / diff(rho, x) + diff(mz, y) / diff(rho, y)

Frho = diff(mz, z) / rho
Fphi = dmz_drho * diff(beta, z) - diff(mz, z) * dbeta_drho

Fx = Frho * cos(phi) - Fphi * sin(phi)
Fy = Frho * sin(phi) + Fphi * cos(phi)
Fz = -dmz_drho / rho

Fx = Fx.simplify()
Fy = Fy.simplify()
Fz = Fz.simplify()

F = vector([Fx, Fy, Fz])

Arho = -(1 + mz) * dbeta_drho
Aphi = (1 - mz) / rho

Ax_coulomb = Arho * cos(phi) - Aphi * sin(phi)
Ay_coulomb = Arho * sin(phi) + Aphi * cos(phi)
Az_coulomb = -(1 + mz) * diff(beta, z)

A_coulomb= vector([Ax_coulomb, Ay_coulomb, Az_coulomb])

Hopf_density_coulomb = A_coulomb.dot_product(F)

Now we calculate the Hopf index density using my gauge. I reformulate the stereographic projection as

\begin{align}
    m_x &= 2(\psi_1 \psi_4 + \psi_2 \psi_3), \\
    m_y &= 2(\psi_1 \psi_3 - \psi_2 \psi_4), \\
    m_z &= \psi_3^2 + \psi_4^2 - \psi_1^2 - \psi_2^2
\end{align}

with

\begin{align}
    \psi_1 &= \frac{2x}{r^2 + 1}, \\
    \psi_2 &= \frac{2y}{r^2 + 1}, \\
    \psi_3 &= \frac{2z}{r^2 + 1}, \\
    \psi_4 &= \frac{r^2 - 1}{r^2 + 1}
\end{align}

which is the same as before, but Sage struggles with the previous formulation.

In [4]:
# Declare variables
x = var('x')
y = var('y')
z = var('z')
r = sqrt(x^2 + y^2 + z^2)

# Hopfion magnetisation ansatz from stereographic projection
psi_1 = 2*x / (r^2 + 1)
psi_2 = 2*y / (r^2 + 1)
psi_3 = 2*z / (r^2 + 1)
psi_4 = (r^2 - 1) / (r^2 + 1)

m_x_ours = 2 * (psi_1 * psi_4 + psi_2 * psi_3)
m_y_ours = 2 * (psi_1 * psi_3 - psi_2 * psi_4)
m_z_ours = psi_3^2 + psi_4^2 - psi_1^2 - psi_2^2

m_ours = vector([m_x_ours, m_y_ours, m_z_ours])

dm_dx = diff(m_ours, x).simplify()
dm_dy = diff(m_ours, y).simplify()
dm_dz = diff(m_ours, z).simplify()

F_x_ours = 2 * m_ours.dot_product(dm_dy.cross_product(dm_dz))
F_y_ours = 2 * m_ours.dot_product(dm_dz.cross_product(dm_dx))
F_z_ours = 2 * m_ours.dot_product(dm_dx.cross_product(dm_dy))
F_ours = vector([F_x_ours, F_y_ours, F_z_ours])

y_prime = var('yprime', latex_name='y^\prime')
F_x_yprime = F_x_ours.subs(y=yprime)
F_z_yprime = F_z_ours.subs(y=yprime)
A_x_ours = integrate(F_z_yprime, yprime, -oo, y).simplify_full()
A_y_ours = 0
A_z_ours = -integrate(F_x_yprime, yprime, -oo, y).simplify_full()
A_ours = vector([A_x_ours, A_y_ours, A_z_ours])

Hopf_density_ours = A_ours.dot_product(F_ours)

Here, we plot the Hopf index density in the $z = 0$ plane. We see that there is circular symmetry for the Coulomb gauge, but in my gauge, it is kind of egg-shaped.

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)

max_hopf_density = 600

contour_plot(
    Hopf_density_ours(z=1e-6),
    (x, -2, 2),
    (y, -2, 2),
    fill=False,
    options={},
    cmap='Reds',
    axes_labels=['$x$', '$y$'],
    vmin=0,
    vmax=max_hopf_density
).matplotlib(sub=ax1)

contour_plot(
    Hopf_density_coulomb(z=1e-6),
    (x, -2, 2),
    (y, -2, 2),
    fill=False,
    options={},
    cmap='Reds',
    axes_labels=['$x$', '$y$'],
    vmin=0,
    vmax=max_hopf_density
).matplotlib(sub=ax2)

ax1.set_xlabel(r'$x$', loc='right')
ax1.set_ylabel(r'$y$', loc='top', rotation=0)
ax2.set_xlabel(r'$x$', loc='right')
ax2.set_ylabel(r'$y$', loc='top', rotation=0)

ax1.set_aspect('equal')
ax2.set_aspect('equal')

ax1.set_xticks([])
ax1.set_yticks([])
ax2.set_xticks([])
ax2.set_yticks([])

plt.savefig('GaugeComparison.pdf', format='pdf', bbox_inches='tight', transparent=True)